<a href="https://colab.research.google.com/github/mohansameer1983/ComputerVisionProjects/blob/main/Trash_Classification_CV.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow.keras as keras
import pandas as pd

import numpy as np
import tensorflow.keras as keras
from tensorflow.keras.layers import Dense, Input, Dropout

     |████████████████████████████████| 3.5 MB 4.3 MB/s 
     |████████████████████████████████| 4.2 MB 25.5 MB/s 
     |████████████████████████████████| 596 kB 38.2 MB/s 
     |████████████████████████████████| 6.8 MB 37.4 MB/s 
     |████████████████████████████████| 895 kB 48.0 MB/s 
     |████████████████████████████████| 67 kB 3.3 MB/s 
     |████████████████████████████████| 76 kB 4.4 MB/s 


Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/483 [00:00<?, ?B/s]

# Data Preparation

In [ ]:
def load_data(save_dir="./"):
  dataset = tfds.load('glue/cola', shuffle_files=True)
  train = tfds.as_dataframe(dataset["train"])
  val = tfds.as_dataframe(dataset["validation"])
  test = tfds.as_dataframe(dataset["test"])
  return train, val, test

def prepare_raw_data(df):
  raw_data = df.loc[:, ["idx", "sentence", "label"]]
  raw_data["label"] = raw_data["label"].astype('category')
  return raw_data

train, val, test = load_data()
train = prepare_raw_data(train)
val = prepare_raw_data(val)
test = prepare_raw_data(test)

num_classes=len(train.label.unique())

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

Generating splits...:   0%|          | 0/3 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/8551 [00:00<?, ? examples/s]

...

Generating validation examples...:   0%|          | 0/1043 [00:00<?, ? examples/s]

Shuffling ~/tensorflow_datasets/glue/cola/2.0.0.incompleteUUC5NE/glue-validation.tfrecord*...:   0%|          …

Generating test examples...:   0%|          | 0/1063 [00:00<?, ? examples/s]

Shuffling ~/tensorflow_datasets/glue/cola/2.0.0.incompleteUUC5NE/glue-test.tfrecord*...:   0%|          | 0/10…

Dataset glue downloaded and prepared to ~/tensorflow_datasets/glue/cola/2.0.0. Subsequent calls will reuse this data.


In [ ]:
def clean_data(df):

  
  return df

print(train)
train = clean_data(train)
val = clean_data(val)
test = clean_data(test)

print(train.head())
print(test.head())

       idx                                           sentence label
0     1680  b'It is this hat that it is certain that he wa...     1
1     1456  b'Her efficient looking up of the answer pleas...     1
2     4223          b'Both the workers will wear carnations.'     1
3     4093  b'John enjoyed drawing trees for his syntax ho...     1
4     7111  b'We consider Leslie rather foolish, and Lou a...     1
...    ...                                                ...   ...
8546  4920              b'a pencil with that to write broke.'     0
8547  3619  b'It was in the park last night that the polic...     0
8548  3624       b'John looked up the inside of the chimney.'     1
8549  8351                     b'You said she liked yourself'     0
8550  5073    b"What we're using is their teaching material."     1

[8551 rows x 3 columns]


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


    idx                                           sentence label
0  1680  b'it this hat that it certain that he was wear...     1
1  1456  b'her efficient looking up of answer pleased b...     1
2  4223                    b'both workers wear carnations'     1
3  4093  b'john enjoyed drawing trees for his syntax ho...     1
4  7111  b'we consider leslie rather foolish lou comple...     1
    idx                                       sentence label
0   163               b'brian was wiping behind stove'    -1
1   131          b'you could give headache to tylenol'    -1
2  1021                         b'i want to meet at 6'    -1
3   166                       b'packages carry easily'    -1
4  1039  b'many people said they were sick who werent'    -1


In [ ]:
def extract_text_and_y(df):
  text = [x.decode('utf-8') for x in  df.sentence.values]
  # for multiclass problems, you can use sklearn.preprocessing.OneHotEncoder, but we only have two classes, so we'll use a single sigmoid output
  y = np.array([x for x in df.label.values])
  return text, y

# the following prepares the input for running
train_text, train_y = extract_text_and_y(clean_data(train))
val_text, val_y = extract_text_and_y(clean_data(val))
test_text, test_y = extract_text_and_y(clean_data(test))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


# Modelling

## Build and Train Model


In [ ]:
def build_model(base_model, trainable=False, params={}):
   
    base_model.trainable = trainable


    # Any additional layers should go here
    # use the 'params' as a dictionary for hyper parameter to facilitate experimentation
    dense_layer = Dense(params['dense_layer_1_units'],activation='relu')(dbert_last_hidden_state[:, 0, :])
    dropout_layer = Dropout(params['dropout_rate_1'])(dense_layer)
    dense_layer = Dense(params['dense_layer_2_units'],activation='relu')(dropout_layer)
    #dropout_layer = Dropout(params['dropout_rate_2'])(dense_layer)
    probs = Dense(1, activation='sigmoid',kernel_regularizer=regularizers.l2(params['regularizer_l2_rate']), name="output")(dense_layer)

    model = keras.Model(inputs=[inputs, masks], outputs=probs)
    model.summary()
    return model

dbert_model = TFDistilBertModel.from_pretrained('distilbert-base-uncased')
params={# add parameters here
        "dense_layer_1_units": 128,
	      "dense_layer_2_units": 64,
        "dropout_rate_1": 0.1,
        "dropout_rate_2": 0.1,
        "regularizer_l2_rate":0.01
        }

model = build_model(dbert_model, params=params)

Downloading:   0%|          | 0.00/347M [00:00<?, ?B/s]

Some layers from the model checkpoint at distilbert-base-uncased were not used when initializing TFDistilBertModel: ['vocab_projector', 'activation_13', 'vocab_layer_norm', 'vocab_transform']
- This IS expected if you are initializing TFDistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFDistilBertModel were initialized from the model checkpoint at distilbert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFDistilBertModel for predictions without further training.


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 24)]         0           []                               
                                                                                                  
 input_2 (InputLayer)           [(None, 24)]         0           []                               
                                                                                                  
 tf_distil_bert_model (TFDistil  TFBaseModelOutput(l  66362880   ['input_1[0][0]',                
 BertModel)                     ast_hidden_state=(N               'input_2[0][0]']                
                                one, 24, 768),                                                    
                                 hidden_states=None                                           

In [ ]:
def compile_model(model):
    # TODO: compile the model, include relevant auc metrics when training
    # DO NOT CHANGE THE INPUTS OR OUTPUTS TO THIS FUNCTION

    METRICS = [
          tf.keras.metrics.BinaryAccuracy(name='accuracy'),
          tf.keras.metrics.Precision(name='precision'),
          tf.keras.metrics.Recall(name='recall')
    ]
    optimizer = keras.optimizers.Adam(learning_rate=1e-5)
    loss = keras.losses.binary_crossentropy
    model.compile(optimizer=optimizer,
              loss=loss,
              metrics=METRICS)
    return model

model = compile_model(model)

In [ ]:
import tensorflow as tf
def train_model(model, model_inputs_and_masks_train, model_inputs_and_masks_val,
    y_train, y_val, batch_size, num_epochs):
    # TODO: train the model
    # DO NOT CHANGE THE INPUTS OR OUTPUTS TO THIS FUNCTION
  
    history = model.fit(dataset,
          epochs=num_epochs,
          batch_size=batch_size, verbose=1)
    
    return model, history

model, history = train_model(model, train_model_inputs_and_masks, val_model_inputs_and_masks, train_y, val_y, batch_size=128, num_epochs=5)

Epoch 1/5
1/1 [==============================] - 352s 352s/step - loss: 0.6849 - accuracy: 0.6423 - precision: 0.7025 - recall: 0.8536
Epoch 2/5
1/1 [==============================] - 350s 350s/step - loss: 0.6822 - accuracy: 0.6487 - precision: 0.7043 - recall: 0.8639
Epoch 3/5
1/1 [==============================] - 350s 350s/step - loss: 0.6817 - accuracy: 0.6494 - precision: 0.7038 - recall: 0.8673
Epoch 4/5
1/1 [==============================] - 348s 348s/step - loss: 0.6812 - accuracy: 0.6572 - precision: 0.7040 - recall: 0.8859
Epoch 5/5
1/1 [==============================] - 352s 352s/step - loss: 0.6785 - accuracy: 0.6550 - precision: 0.7015 - recall: 0.8883
